<a href="https://colab.research.google.com/github/marija1023/cytotoxic-t-lymphocytes-classification/blob/master/ip2_proj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import LocalOutlierFactor


In [0]:
df_1 = pd.read_csv("drive/My Drive/033_Precursors of human CD4+ cytotoxic T lymphocytes_csv.csv")
df_2 = pd.read_csv("drive/My Drive/034_Precursors of human CD4+ cytotoxic T lymphocytes_csv.csv")

In [0]:
print(df_1.head(10))

       Unnamed: 0  1  2  3  4  5  6  ...  3773  3774  3775  3776  3777  3778  3779
0       hg38_A1BG  0  0  0  0  0  0  ...     0     0     0     0     0     0     0
1   hg38_A1BG-AS1  0  0  0  0  0  0  ...     0     0     0     0     0     0     0
2       hg38_A1CF  0  0  0  0  0  0  ...     0     0     0     0     0     0     0
3        hg38_A2M  0  0  0  0  1  0  ...     0     1     0     0     0     0     0
4    hg38_A2M-AS1  1  0  0  0  1  0  ...     0     0     0     0     0     0     0
5      hg38_A2ML1  0  0  0  0  0  0  ...     0     0     0     0     0     0     0
6  hg38_A2ML1-AS1  0  0  0  0  0  0  ...     0     0     0     0     0     0     0
7  hg38_A2ML1-AS2  0  0  0  0  0  0  ...     0     0     0     0     0     0     0
8    hg38_A3GALT2  0  0  0  0  0  0  ...     0     0     0     0     0     0     0
9     hg38_A4GALT  0  0  0  0  0  0  ...     0     0     0     0     0     0     0

[10 rows x 3780 columns]


# **Pre-processing**

In [0]:
df_1 = df_1.transpose()
df_2 = df_2.transpose()

In [0]:
print(df_1.head(10))

                0              1      ...     31219       31220
Unnamed: 0  hg38_A1BG  hg38_A1BG-AS1  ...  hg38_ZYX  hg38_ZZEF1
1                   0              0  ...         0           0
2                   0              0  ...         0           0
3                   0              0  ...         0           0
4                   0              0  ...         0           0
5                   0              0  ...         0           0
6                   0              0  ...         0           0
7                   0              0  ...         0           0
8                   0              0  ...         0           0
9                   0              0  ...         0           0

[10 rows x 31221 columns]


In [0]:
print(df_1.shape)
print(df_2.shape)

(3780, 31221)
(3206, 31221)


In [0]:
df_1['class'] = "c033"
# print(df_1.head(5))
df_2['class'] = "c034"

**Filter 0-value columns**

In [0]:
# Proverava se da li su obe kolone jednog atributa iz razlicitih klasa razlicite od 0
# Ako je barem jedna kolona razlicita od 0, tada se obe kolone dodaju u spojenu tabelu

df = pd.DataFrame()
for column in df_1:
  col1 = df_1[column][1:]
  col2 = df_2[column][1:]
  col_name = df_1[column][0]
  if np.any(col1) or np.any(col2):
    df[df_1[column][0]] = col1.append(col2)
    
df.rename(columns = {"c033" : "class"}, inplace = True)
    
# df.to_csv("after_preprocessing.csv", sep=',', encoding="utf-8", index=False)

In [0]:
print(df.shape)
print(df.head(10))

(6984, 16954)
   hg38_A1BG hg38_A1BG-AS1 hg38_A2M  ... hg38_ZYX hg38_ZZEF1 class
1          0             0        0  ...        0          0  c033
2          0             0        0  ...        0          0  c033
3          0             0        0  ...        0          0  c033
4          0             0        0  ...        0          0  c033
5          0             0        1  ...        0          0  c033
6          0             0        0  ...        0          0  c033
7          0             0        0  ...        0          0  c033
8          0             0        0  ...        0          0  c033
9          0             0        0  ...        0          0  c033
10         0             0        0  ...        0          0  c033

[10 rows x 16954 columns]


In [0]:
df_without_nan = df.dropna(axis=0, how='any')
print(df_without_nan.shape)
print(df.shape)

(6984, 16954)
(6984, 16954)


In [0]:
import pandas as pd
df_or = pd.read_csv("drive/My Drive/after_preprocessing_or.csv")

In [0]:
df_inf = df_or.replace([np.inf, -np.inf], np.nan)
df_inf = df_or.dropna(axis=0, how='any')
print(df_inf.shape)
print(df_or.shape)

(6984, 16954)
(6984, 16954)


**Outliers**

In [0]:
df_or.replace("c033", 33, inplace = True)
df_or.replace("c034", 34, inplace = True)
lof = LocalOutlierFactor(n_neighbors=10, metric='minkowski')
lof.fit(df_or)
lof_factor = lof.negative_outlier_factor_

/usr/local/lib/python3.6/dist-packages/sklearn/neighbors/lof.py:236: FutureWarning: default contamination parameter 0.1 will change in version 0.22 to "auto". This will change the predict method behavior.
  FutureWarning)


In [0]:
print(lof_factor)

[-1.08176683 -1.01484019 -1.04819681 ... -1.08865598 -1.07991024
 -1.11904618]


In [0]:
df_nan = df_or.dropna(axis=0, how='any')
print(df_nan.shape)

(0, 16954)


In [0]:
outlier_factor = 1.09

outlier_points = df_or[lof_factor < -outlier_factor]
inlier_points = df_or[lof_factor >= -outlier_factor]
print(outlier_points.shape)
print(inlier_points.shape)
# df.filter(df[lof_factor < -outlier_factor])
print(df_or.shape)

(1112, 16954)
(5872, 16954)
(6984, 16954)


In [0]:
inlier_points.to_csv("inlier_points.csv", sep=',', encoding="utf-8", index=False)

In [0]:
outlier_points.to_csv("outlier_points.csv", sep=',', encoding="utf-8", index=False)

In [0]:
outlier_points = pd.read_csv("drive/My Drive/outlier_points.csv")
inlier_points = pd.read_csv("drive/My Drive/inlier_points.csv")

In [0]:
inlier_points.rename(columns = {"c033" : "class"}, inplace = True)
outlier_points.rename(columns = {"c033" : "class"}, inplace = True)

In [0]:
inlier_points.replace("c033", 33, inplace = True)
inlier_points.replace("c034", 34, inplace = True)

outlier_points.replace("c033", 33, inplace = True)
outlier_points.replace("c034", 34, inplace = True)

# **Feature  selection**

In [0]:
# Feature selection

# Removing features with low variance
# Var[X] = p(1-p)
from sklearn.feature_selection import VarianceThreshold

sel = VarianceThreshold(threshold=(.8 * (1 - .8)))
X = df_or.iloc[:, :-1].values
sel.fit_transform(X)

array([[1, 2, 0, ..., 0, 0, 2],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 2, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [1, 3, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1]])

In [0]:
# Univariate feature selection
# Statistical test can be used to select those features that have the
# strongest relationship with the outpu variable
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

X = df_or.iloc[:, :-1].values
y = np.transpose(df_or.iloc[:, [-1]].values)[0]

# feature extraction
test = SelectKBest(score_func=chi2, k=4)
fit = test.fit(X, y)
# summarize scores
np.set_printoptions(precision=3)
print(fit.scores_)
# features = fit.transform(X)
# summarize selected features
# print(features[0:5,:])

[49.666  5.516 17.786 ...  8.443 10.359  5.703]


In [0]:
print(list(df_or))
# print(min(fit.scores_))

['hg38_A1BG', 'hg38_A1BG-AS1', 'hg38_A2M', 'hg38_A2M-AS1', 'hg38_A2ML1', 'hg38_AAAS', 'hg38_AACS', 'hg38_AAED1', 'hg38_AAGAB', 'hg38_AAK1', 'hg38_AAMDC', 'hg38_AAMP', 'hg38_AAR2', 'hg38_AARD', 'hg38_AARS', 'hg38_AARS2', 'hg38_AARSD1', 'hg38_AASDH', 'hg38_AASDHPPT', 'hg38_AASS', 'hg38_AATBC', 'hg38_ABAT', 'hg38_ABCA1', 'hg38_ABCA10', 'hg38_ABCA2', 'hg38_ABCA3', 'hg38_ABCA5', 'hg38_ABCA6', 'hg38_ABCA7', 'hg38_ABCB1', 'hg38_ABCB10', 'hg38_ABCB4', 'hg38_ABCB5', 'hg38_ABCB7', 'hg38_ABCB8', 'hg38_ABCB9', 'hg38_ABCC1', 'hg38_ABCC10', 'hg38_ABCC11', 'hg38_ABCC2', 'hg38_ABCC3', 'hg38_ABCC4', 'hg38_ABCC5', 'hg38_ABCC6', 'hg38_ABCD1', 'hg38_ABCD2', 'hg38_ABCD3', 'hg38_ABCD4', 'hg38_ABCE1', 'hg38_ABCF1', 'hg38_ABCF2', 'hg38_ABCF3', 'hg38_ABCG1', 'hg38_ABCG2', 'hg38_ABHD1', 'hg38_ABHD10', 'hg38_ABHD11', 'hg38_ABHD11-AS1', 'hg38_ABHD12', 'hg38_ABHD13', 'hg38_ABHD14A', 'hg38_ABHD14B', 'hg38_ABHD15', 'hg38_ABHD15-AS1', 'hg38_ABHD16A', 'hg38_ABHD16B', 'hg38_ABHD17A', 'hg38_ABHD17B', 'hg38_ABHD17C', 'hg

In [0]:
attrs_scores = list(zip(list(df_or), fit.scores_))
print(attrs_scores)
print("Best:")
best = max(attrs_scores, key=lambda x: x[1])
score_best = best[1]
genome_best = best[0]
print(genome_best, score_best)

worst = min(attrs_scores, key=lambda x: x[1])
score_worst = worst[1]
genome_worst = worst[0]
print("Worst:")
print(genome_worst, score_worst)

[('hg38_A1BG', 49.666394162424865), ('hg38_A1BG-AS1', 5.515633266618988), ('hg38_A2M', 17.785922133096264), ('hg38_A2M-AS1', 0.8594184660714623), ('hg38_A2ML1', 1.1790951638065525), ('hg38_AAAS', 6.206069221660869), ('hg38_AACS', 2.171113588973302), ('hg38_AAED1', 12.368010002104498), ('hg38_AAGAB', 15.038948737390346), ('hg38_AAK1', 20.51144406225952), ('hg38_AAMDC', 4.104465137680095), ('hg38_AAMP', 1.373266415010471), ('hg38_AAR2', 3.3652741842193254), ('hg38_AARD', 0.013601564438338314), ('hg38_AARS', 3.1622718680691966), ('hg38_AARS2', 2.527735328575028), ('hg38_AARSD1', 6.683629995636452), ('hg38_AASDH', 0.5163312319208835), ('hg38_AASDHPPT', 9.60752626812349), ('hg38_AASS', 6.784863720560995), ('hg38_AATBC', 0.0816093866300299), ('hg38_ABAT', 9.592793145090642), ('hg38_ABCA1', 12.970046801872076), ('hg38_ABCA10', 0.22681945012651), ('hg38_ABCA2', 2.0210749831639525), ('hg38_ABCA3', 1.8926223900066037), ('hg38_ABCA5', 17.911225115131074), ('hg38_ABCA6', 1.042992413530889), ('hg38

# **Classification**

In [0]:
from sklearn.model_selection import train_test_split
import numpy as np

In [0]:
A = df_or.iloc[:, :-1].values
b = np.transpose(df_or.iloc[:, [-1]].values)[0]

A_train, A_test, b_train, b_test = train_test_split(A, b, stratify=b, test_size=0.3)

In [0]:
A = inlier_points.iloc[:, :-1].values
b = np.transpose(inlier_points.iloc[:, [-1]].values)[0]

A_train, A_test, b_train, b_test = train_test_split(A, b, stratify=b, test_size=0.3)

In [0]:
X = outlier_points.iloc[:, :-1].values
y = np.transpose(outlier_points.iloc[:, [-1]].values)[0]

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.3)

In [0]:
print(df_or.dtypes)

**SVM algorithm**

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, classification_report

In [0]:
svm_clf = SVC(gamma='auto')
svm_clf.fit(A_train, b_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [0]:
b_test_predicted_svm = svm_clf.predict(A_test)

In [0]:
print("SVM test score:")
print(svm_clf.score(A_test, b_test))

SVM test score:
0.9875141884222475


In [0]:
b_train_predicted_svm = svm_clf.predict(A_train)

In [0]:
print("SVM train score:")
print(svm_clf.score(A_train, b_train))

SVM train score:
0.991727493917275


In [0]:
test_rep_svm = classification_report(b_test, b_test_predicted_svm)
print("SVM test report:\n{}".format(test_rep_svm))

SVM test report:
              precision    recall  f1-score   support

          33       0.98      1.00      0.99       977
          34       0.99      0.98      0.99       785

    accuracy                           0.99      1762
   macro avg       0.99      0.99      0.99      1762
weighted avg       0.99      0.99      0.99      1762



In [0]:
train_rep_svm = classification_report(b_train, b_train_predicted_svm)
print("SVM train report:\n{}".format(train_rep_svm))

SVM train report:
              precision    recall  f1-score   support

          33       0.99      1.00      0.99      2279
          34       1.00      0.98      0.99      1831

    accuracy                           0.99      4110
   macro avg       0.99      0.99      0.99      4110
weighted avg       0.99      0.99      0.99      4110



In [0]:
train_conf_svm = confusion_matrix(b_train, b_train_predicted_svm)
print("Confusion matrix - train:\n{}".format(train_conf_svm))

Confusion matrix - train:
[[2277    2]
 [  32 1799]]


In [0]:
test_conf_svm = confusion_matrix(b_test, b_test_predicted_svm)
print("Confusion matrix - test:\n{}".format(test_conf_svm))

Confusion matrix - test:
[[973   4]
 [ 18 767]]


In [0]:
svm_clf_out = SVC(gamma='auto')
svm_clf_out.fit(X_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [0]:
y_test_predicted_svm = svm_clf_out.predict(X_test)

In [0]:
print("SVM test score for outliers:")
print(svm_clf_out.score(X_test, y_test))

SVM test score for outliers:
0.9461077844311377


In [0]:
y_train_predicted_svm = svm_clf_out.predict(X_train)

In [0]:
print("SVM train score for outliers:")
print(svm_clf_out.score(X_train, y_train))

SVM train score for outliers:
0.967866323907455


In [0]:
test_rep_svm_out = classification_report(y_test, y_test_predicted_svm)
print("SVM test report for outliers:\n{}".format(test_rep_svm_out))

SVM test report for outliers:
              precision    recall  f1-score   support

          33       0.94      0.95      0.94       157
          34       0.95      0.94      0.95       177

    accuracy                           0.95       334
   macro avg       0.95      0.95      0.95       334
weighted avg       0.95      0.95      0.95       334



In [0]:
train_rep_svm_out = classification_report(y_train, y_train_predicted_svm)
print("SVM train report for outliers:\n{}".format(train_rep_svm_out))

SVM train report for outliers:
              precision    recall  f1-score   support

          33       0.97      0.96      0.97       366
          34       0.96      0.98      0.97       412

    accuracy                           0.97       778
   macro avg       0.97      0.97      0.97       778
weighted avg       0.97      0.97      0.97       778



In [0]:
train_conf_svm_out = confusion_matrix(y_train, y_train_predicted_svm)
print("Confusion matrix for outliers - train:\n{}".format(train_conf_svm_out))

Confusion matrix for outliers - train:
[[351  15]
 [ 10 402]]


In [0]:
test_conf_svm_out = confusion_matrix(y_test, y_test_predicted_svm)
print("Confusion matrix for outliers - test:\n{}".format(test_conf_svm_out))

Confusion matrix for outliers - test:
[[149   8]
 [ 10 167]]


**Logistic regression**

In [0]:
from sklearn.linear_model import LogisticRegression

In [0]:
lr_clf = LogisticRegression(C=100000000000.0, penalty='l1')
lr_clf.fit(A_train, b_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=100000000000.0, class_weight=None, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='warn', n_jobs=None, penalty='l1',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [0]:
b_test_predicted_lr = lr_clf.predict(A_test)

In [0]:
print("Logistic Regression test score:")
print(lr_clf.score(A_test, b_test))

Logistic Regression test score:
0.9148694665153235


In [0]:
b_train_predicted_lr = lr_clf.predict(A_train)

In [0]:
print("Logistic Regression train score:")
print(lr_clf.score(A_train, b_train))

Logistic Regression train score:
1.0


In [0]:
test_rep_lr = classification_report(b_test, b_test_predicted_lr)
print("Logistic Regression test report:\n{}".format(test_rep_lr))

Logistic Regression test report:
              precision    recall  f1-score   support

          33       0.99      0.99      0.99       977
          34       0.99      0.99      0.99       785

    accuracy                           0.99      1762
   macro avg       0.99      0.99      0.99      1762
weighted avg       0.99      0.99      0.99      1762



In [0]:
train_rep_lr = classification_report(b_train, b_train_predicted_lr)
print("Logistic Regression train report:\n{}".format(train_rep_lr))

Logistic Regression train report:
              precision    recall  f1-score   support

          33       1.00      1.00      1.00      2279
          34       1.00      1.00      1.00      1831

    accuracy                           1.00      4110
   macro avg       1.00      1.00      1.00      4110
weighted avg       1.00      1.00      1.00      4110



In [0]:
test_conf_lr = confusion_matrix(b_test, b_test_predicted_lr)
print("Confusion matrix test:\n{}".format(test_conf_lr))

Confusion matrix test:
[[970   7]
 [  5 780]]


In [0]:
train_conf_lr = confusion_matrix(b_train, b_train_predicted_lr)
print("Confusion matrix train:\n{}".format(train_conf_lr))

Confusion matrix train:
[[2279    0]
 [   0 1831]]


In [0]:
lr_clf_out = LogisticRegression()
lr_clf_out.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [0]:
y_test_predicted_lr = lr_clf_out.predict(X_test)

In [0]:
print("Logistic Regression test score for outliers:")
print(lr_clf_out.score(X_test, y_test))

Logistic Regression test score for outliers:
0.9820359281437125


In [0]:
y_train_predicted_lr = lr_clf_out.predict(X_train)

In [0]:
print("Logistic Regression train score for outliers:")
print(lr_clf_out.score(X_train, y_train))

Logistic Regression test score for outliers:
1.0


In [0]:
test_rep_lr_out = classification_report(y_test, y_test_predicted_lr)
print("Logistic Regresion test report for outliers:\n{}".format(test_rep_lr_out))

Logistic Regresion test report for outliers:
              precision    recall  f1-score   support

          33       0.97      0.99      0.98       157
          34       0.99      0.97      0.98       177

    accuracy                           0.98       334
   macro avg       0.98      0.98      0.98       334
weighted avg       0.98      0.98      0.98       334



In [0]:
train_rep_lr_out = classification_report(y_train, y_train_predicted_lr)
print("Logistic Regression train report for outliers:\n{}".format(train_rep_lr_out))

Logistic Regression train report for outliers:
              precision    recall  f1-score   support

          33       1.00      1.00      1.00       366
          34       1.00      1.00      1.00       412

    accuracy                           1.00       778
   macro avg       1.00      1.00      1.00       778
weighted avg       1.00      1.00      1.00       778



In [0]:
train_conf_lr_out = confusion_matrix(y_train, y_train_predicted_lr)
print("Confusion matrix for outliers - train:\n{}".format(train_conf_lr_out))

Confusion matrix for outliers - train:
[[366   0]
 [  0 412]]


In [0]:
test_conf_lr_out = confusion_matrix(y_test, y_test_predicted_lr)
print("Confusion matrix for outliers - test:\n{}".format(test_conf_lr_out))

Confusion matrix for outliers - test:
[[156   1]
 [  5 172]]


**SGD algorithm**

In [0]:
from sklearn.linear_model import SGDClassifier

In [0]:
sgd_clf = SGDClassifier(loss="hinge", penalty="l2", max_iter=5)
sgd_clf.fit(A_train, b_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/stochastic_gradient.py:561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)


SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=5,
              n_iter_no_change=5, n_jobs=None, penalty='l2', power_t=0.5,
              random_state=None, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [0]:
b_test_predicted_sgd = sgd_clf.predict(A_test)

In [0]:
print("SGD score test:")
print(sgd_clf.score(A_test, b_test))

SGD score test:
0.9869466515323496


In [0]:
b_train_predicted_sgd = sgd_clf.predict(A_train)

In [0]:
print("SGD score train:")
print(sgd_clf.score(A_train, b_train))

SGD score train:
0.9931873479318735


In [0]:
test_rep_sgd = classification_report(b_test, b_test_predicted_sgd)
print("SGD report test:\n{}".format(test_rep_sgd))

SGD report test:
              precision    recall  f1-score   support

          33       0.99      0.98      0.99       977
          34       0.98      0.99      0.99       785

    accuracy                           0.99      1762
   macro avg       0.99      0.99      0.99      1762
weighted avg       0.99      0.99      0.99      1762



In [0]:
train_rep_sgd = classification_report(b_train, b_train_predicted_sgd)
print("SGD report train:\n{}".format(train_rep_sgd))

SGD report train:
              precision    recall  f1-score   support

          33       1.00      0.99      0.99      2279
          34       0.99      0.99      0.99      1831

    accuracy                           0.99      4110
   macro avg       0.99      0.99      0.99      4110
weighted avg       0.99      0.99      0.99      4110



In [0]:
test_conf_sgd = confusion_matrix(b_test, b_test_predicted_sgd)
print("Confusion matrix - test:\n{}".format(test_conf_sgd))

Confusion matrix:
[[1125    9]
 [  14  948]]


In [0]:
train_conf_sgd = confusion_matrix(b_train, b_train_predicted_sgd)
print("Confusion matrix - train:\n{}".format(train_conf_sgd))

Confusion matrix - train:
[[2262   17]
 [  11 1820]]


In [0]:
sgd_clf_out = SGDClassifier(loss="hinge", penalty="l2", max_iter=2)
# za max_iter=5 daje preciznost 0.54, za 3 daje 0.94
sgd_clf_out.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/stochastic_gradient.py:561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)


SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=2,
              n_iter_no_change=5, n_jobs=None, penalty='l2', power_t=0.5,
              random_state=None, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [0]:
y_test_predicted_sgd = sgd_clf_out.predict(X_test)

In [0]:
print("SGD test score for outliers:")
print(sgd_clf_out.score(X_test, y_test))

SGD test score for outliers:
0.9520958083832335


In [0]:
y_train_predicted_sgd = sgd_clf_out.predict(X_train)

In [0]:
print("SGD train score for outliers:")
print(sgd_clf_out.score(X_train, y_train))

SGD test score for outliers:
0.9704370179948586


In [0]:
test_rep_sgd_out = classification_report(y_test, y_test_predicted_sgd)
print("SGD test report for outliers:\n{}".format(test_rep_sgd_out))

SGD test report for outliers:
              precision    recall  f1-score   support

          33       0.91      0.99      0.95       157
          34       0.99      0.92      0.95       177

    accuracy                           0.95       334
   macro avg       0.95      0.95      0.95       334
weighted avg       0.96      0.95      0.95       334



In [0]:
train_rep_sgd_out = classification_report(y_train, y_train_predicted_sgd)
print("SGD train report for outliers:\n{}".format(train_rep_sgd_out))

SGD train report for outliers:
              precision    recall  f1-score   support

          33       0.96      0.98      0.97       366
          34       0.98      0.96      0.97       412

    accuracy                           0.97       778
   macro avg       0.97      0.97      0.97       778
weighted avg       0.97      0.97      0.97       778



In [0]:
train_conf_sgd_out = confusion_matrix(y_train, y_train_predicted_sgd)
print("Confusion matrix for outliers - train:\n{}".format(train_conf_sgd_out))

Confusion matrix for outliers - train:
[[359   7]
 [ 16 396]]


In [0]:
test_conf_sgd_out = confusion_matrix(y_test, y_test_predicted_sgd)
print("Confusion matrix for outliers - test:\n{}".format(test_conf_sgd_out))

Confusion matrix for outliers - test:
[[156   1]
 [ 15 162]]


**KNN algorithm**

In [0]:
from sklearn.neighbors import KNeighborsClassifier

In [0]:
knn_clf = KNeighborsClassifier(n_neighbors=3)
knn_clf.fit(A_train, b_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                     weights='uniform')

In [0]:
b_test_predicted_knn = knn_clf.predict(A_test)

In [0]:
print("KNN score test:")
print(knn_clf.score(A_test, b_test))

KNN score test:
0.927355278093076


In [0]:
b_train_predicted_knn = knn_clf.predict(A_train)

In [0]:
print("KNN train score:")
print(knn_clf.score(A_train, b_train))

KNN train score:
0.9661800486618005


In [0]:
test_rep_knn = classification_report(b_test, b_test_predicted_knn)
print("KNN test report:\n{}".format(test_rep_knn))

KNN test report:
              precision    recall  f1-score   support

          33       0.90      0.98      0.94       977
          34       0.97      0.87      0.91       785

    accuracy                           0.93      1762
   macro avg       0.93      0.92      0.93      1762
weighted avg       0.93      0.93      0.93      1762



In [0]:
train_rep_knn = classification_report(b_train, b_train_predicted_knn)
print("KNN train report:\n{}".format(train_rep_knn))

KNN train report:
              precision    recall  f1-score   support

          33       0.95      0.99      0.97      2279
          34       0.99      0.94      0.96      1831

    accuracy                           0.97      4110
   macro avg       0.97      0.96      0.97      4110
weighted avg       0.97      0.97      0.97      4110



In [0]:
test_conf_knn = confusion_matrix(b_test, b_test_predicted_knn)
print("Confusion matrix - test:\n{}".format(test_conf_knn))

Confusion matrix - test:
[[954  23]
 [105 680]]


In [0]:
train_conf_knn = confusion_matrix(b_train, b_train_predicted_knn)
print("Confusion matrix - train:\n{}".format(train_conf_knn))

Confusion matrix - train:
[[2258   21]
 [ 118 1713]]


In [0]:
knn_clf_out = KNeighborsClassifier(n_neighbors=7)
knn_clf_out.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=7, p=2,
                     weights='uniform')

In [0]:
y_test_predicted_knn = knn_clf_out.predict(X_test)

In [0]:
print("KNN test score for outliers:")
print(knn_clf_out.score(X_test, y_test))

KNN test score for outliers:
0.8502994011976048


In [0]:
y_train_predicted_knn = knn_clf_out.predict(X_train)

In [0]:
print("KNN train score for outliers:")
print(knn_clf_out.score(X_train, y_train))

KNN train score for outliers:
0.8791773778920309


In [0]:
test_rep_knn_out = classification_report(y_test, y_test_predicted_knn)
print("KNN test report for outliers:\n{}".format(test_rep_knn_out))

KNN test report for outliers:
              precision    recall  f1-score   support

          33       0.81      0.90      0.85       157
          34       0.90      0.81      0.85       177

    accuracy                           0.85       334
   macro avg       0.85      0.85      0.85       334
weighted avg       0.86      0.85      0.85       334



In [0]:
train_rep_knn_out = classification_report(y_train, y_train_predicted_knn)
print("KNN train report for outliers:\n{}".format(train_rep_knn_out))

KNN train report for outliers:
              precision    recall  f1-score   support

          33       0.85      0.90      0.87       366
          34       0.91      0.86      0.88       412

    accuracy                           0.88       778
   macro avg       0.88      0.88      0.88       778
weighted avg       0.88      0.88      0.88       778



In [0]:
train_conf_knn_out = confusion_matrix(y_train, y_train_predicted_knn)
print("Confusion matrix for outliers - train:\n{}".format(train_conf_knn_out))

Confusion matrix for outliers - train:
[[329  37]
 [ 57 355]]


In [0]:
test_conf_knn_out = confusion_matrix(y_test, y_test_predicted_knn)
print("Confusion matrix for outliers - test:\n{}".format(test_conf_knn_out))

Confusion matrix for outliers - test:
[[141  16]
 [ 34 143]]


**Gaussian Naive Bayes**

In [0]:
from sklearn.naive_bayes import GaussianNB

In [0]:
nb_clf = GaussianNB()
nb_clf.fit(A_train, b_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [0]:
b_test_predicted_nb = nb_clf.predict(A_test)

In [0]:
print("Naive Bayes test score:")
print(nb_clf.score(A_test, b_test))

Naive Bayes test score:
0.7156640181611805


In [0]:
b_train_predicted_nb = nb_clf.predict(A_train)

In [0]:
print("Naive Bayes train score:")
print(nb_clf.score(A_train, b_train))

Naive Bayes train score:
0.9474452554744526


In [0]:
test_rep_nb = classification_report(b_test, b_test_predicted_nb)
print("Gaussian Naive Bayes test report:\n{}".format(test_rep_nb))

Gaussian Naive Bayes test report:
              precision    recall  f1-score   support

          33       0.73      0.77      0.75       977
          34       0.69      0.65      0.67       785

    accuracy                           0.72      1762
   macro avg       0.71      0.71      0.71      1762
weighted avg       0.71      0.72      0.71      1762



In [0]:
train_rep_nb = classification_report(b_train, b_train_predicted_nb)
print("Gaussian Naive Bayes train report:\n{}".format(train_rep_nb))

Gaussian Naive Bayes train report:
              precision    recall  f1-score   support

          33       0.92      1.00      0.95      2279
          34       1.00      0.89      0.94      1831

    accuracy                           0.95      4110
   macro avg       0.96      0.94      0.95      4110
weighted avg       0.95      0.95      0.95      4110



In [0]:
test_conf_nb = confusion_matrix(b_test, b_test_predicted_nb)
print("Confusion matrix - test:\n{}".format(test_conf_nb))

Confusion matrix - test:
[[752 225]
 [276 509]]


In [0]:
train_conf_nb = confusion_matrix(b_train, b_train_predicted_nb)
print("Confusion matrix - train:\n{}".format(train_conf_nb))

Confusion matrix - train:
[[2272    7]
 [ 209 1622]]


In [0]:
nb_clf_out = GaussianNB()
nb_clf_out.fit(X_train, y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [0]:
y_test_predicted_nb = nb_clf_out.predict(X_test)

In [0]:
print("Naive Bayes test score for outliers:")
print(nb_clf_out.score(X_test, y_test))

Naive Bayes test score for outliers:
0.6227544910179641


In [0]:
y_train_predicted_nb = nb_clf_out.predict(X_train)

In [0]:
print("Naive Bayes train score for outliers:")
print(nb_clf_out.score(X_train, y_train))

Naive Bayes train score for outliers:
0.993573264781491


In [0]:
test_rep_nb_out = classification_report(y_test, y_test_predicted_nb)
print("Gaussian Naive Bayes test report for outliers:\n{}".format(test_rep_nb_out))

Gaussian Naive Bayes test report for outliers:
              precision    recall  f1-score   support

          33       0.72      0.32      0.44       157
          34       0.60      0.89      0.71       177

    accuracy                           0.62       334
   macro avg       0.66      0.61      0.58       334
weighted avg       0.66      0.62      0.59       334



In [0]:
train_rep_nb_out = classification_report(y_train, y_train_predicted_nb)
print("Gaussian Naive Bayes train report for outliers:\n{}".format(train_rep_nb_out))

Gaussian Naive Bayes train report for outliers:
              precision    recall  f1-score   support

          33       0.99      1.00      0.99       366
          34       1.00      0.99      0.99       412

    accuracy                           0.99       778
   macro avg       0.99      0.99      0.99       778
weighted avg       0.99      0.99      0.99       778



In [0]:
train_conf_nb_out = confusion_matrix(y_train, y_train_predicted_nb)
print("Confusion matrix for outliers - train:\n{}".format(train_conf_nb_out))

Confusion matrix for outliers - train:
[[366   0]
 [  5 407]]


In [0]:
test_conf_nb_out = confusion_matrix(y_test, y_test_predicted_nb)
print("Confusion matrix for outliers - test:\n{}".format(test_conf_nb_out))

Confusion matrix for outliers - test:
[[ 50 107]
 [ 19 158]]


**Decision Tree algorithm**

In [0]:
from sklearn.tree import DecisionTreeClassifier

In [0]:
dt_clf = DecisionTreeClassifier(max_depth=3)
dt_clf.fit(A_train, b_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [0]:
b_test_predicted_dt = dt_clf.predict(A_test)

In [0]:
print("Decision tree test score:")
print(dt_clf.score(A_test, b_test))

Decision tree test score:
0.9812712826333712


In [0]:
b_train_predicted_dt = dt_clf.predict(A_train)

In [0]:
print("Decision tree train score:")
print(dt_clf.score(A_train, b_train))

Decision tree train score:
0.9844282238442822


In [0]:
test_rep_dt = classification_report(b_test, b_test_predicted_dt)
print("Decision tree test report:\n{}".format(test_rep_dt))

Decision tree test report:
              precision    recall  f1-score   support

          33       0.98      0.99      0.98       977
          34       0.99      0.98      0.98       785

    accuracy                           0.98      1762
   macro avg       0.98      0.98      0.98      1762
weighted avg       0.98      0.98      0.98      1762



In [0]:
train_rep_dt = classification_report(b_train, b_train_predicted_dt)
print("Decision tree train report:\n{}".format(train_rep_dt))

Decision tree train report:
              precision    recall  f1-score   support

          33       1.00      1.00      1.00      2279
          34       1.00      0.99      1.00      1831

    accuracy                           1.00      4110
   macro avg       1.00      1.00      1.00      4110
weighted avg       1.00      1.00      1.00      4110



In [0]:
test_conf_dt = confusion_matrix(b_test, b_test_predicted_dt)
print("Confusion matrix:\n{}".format(test_conf_dt))

Confusion matrix:
[[966  11]
 [ 19 766]]


In [0]:
train_conf_dt = confusion_matrix(b_train, b_train_predicted_dt)
print("Confusion matrix:\n{}".format(train_conf_dt))

Confusion matrix:
[[2279    0]
 [  11 1820]]


In [0]:
dt_clf_out = DecisionTreeClassifier(max_depth=3)
dt_clf_out.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [0]:
y_test_predicted_dt = dt_clf_out.predict(X_test)

In [0]:
print("Decision tree test score for outliers:")
print(dt_clf_out.score(X_test, y_test))

Decision tree test score for outliers:
0.9491017964071856


In [0]:
print("Decision tree train score for outliers:")
print(dt_clf_out.score(X_train, y_train))

Decision tree train score for outliers:
0.9781491002570694


In [0]:
y_train_predicted_dt = dt_clf_out.predict(X_train)

In [0]:
test_rep_dt_out = classification_report(y_test, y_test_predicted_dt)
print("Decision tree test report for outliers:\n{}".format(test_rep_dt_out))

Decision tree test report for outliers:
              precision    recall  f1-score   support

          33       0.93      0.95      0.94       157
          34       0.95      0.94      0.95       177

    accuracy                           0.94       334
   macro avg       0.94      0.94      0.94       334
weighted avg       0.94      0.94      0.94       334



In [0]:
train_rep_dt_out = classification_report(y_train, y_train_predicted_dt)
print("Decision tree train report for outliers:\n{}".format(train_rep_dt_out))

Decision tree train report for outliers:
              precision    recall  f1-score   support

          33       0.99      1.00      0.99       366
          34       1.00      0.99      0.99       412

    accuracy                           0.99       778
   macro avg       0.99      0.99      0.99       778
weighted avg       0.99      0.99      0.99       778



In [0]:
train_conf_dt_out = confusion_matrix(y_train, y_train_predicted_dt)
print("Confusion matrix for outliers - train:\n{}".format(train_conf_dt_out))

Confusion matrix for outliers - train:
[[365   1]
 [  4 408]]


In [0]:
test_conf_dt_out = confusion_matrix(y_test, y_test_predicted_dt)
print("Confusion matrix for outliers - test:\n{}".format(test_conf_dt_out))

Confusion matrix for outliers - test:
[[149   8]
 [ 11 166]]


In [0]:
import graphviz
from sklearn import tree

In [0]:
tree.plot_tree(dt_clf.fit(A, b))

In [0]:
feature_names = list(inlier_points)[:-1]
# target_names = list(inlier_points)[-1]
target_names = ["33", "34"]
print(feature_names)
print(target_names)

['hg38_A1BG', 'hg38_A1BG-AS1', 'hg38_A2M', 'hg38_A2M-AS1', 'hg38_A2ML1', 'hg38_AAAS', 'hg38_AACS', 'hg38_AAED1', 'hg38_AAGAB', 'hg38_AAK1', 'hg38_AAMDC', 'hg38_AAMP', 'hg38_AAR2', 'hg38_AARD', 'hg38_AARS', 'hg38_AARS2', 'hg38_AARSD1', 'hg38_AASDH', 'hg38_AASDHPPT', 'hg38_AASS', 'hg38_AATBC', 'hg38_ABAT', 'hg38_ABCA1', 'hg38_ABCA10', 'hg38_ABCA2', 'hg38_ABCA3', 'hg38_ABCA5', 'hg38_ABCA6', 'hg38_ABCA7', 'hg38_ABCB1', 'hg38_ABCB10', 'hg38_ABCB4', 'hg38_ABCB5', 'hg38_ABCB7', 'hg38_ABCB8', 'hg38_ABCB9', 'hg38_ABCC1', 'hg38_ABCC10', 'hg38_ABCC11', 'hg38_ABCC2', 'hg38_ABCC3', 'hg38_ABCC4', 'hg38_ABCC5', 'hg38_ABCC6', 'hg38_ABCD1', 'hg38_ABCD2', 'hg38_ABCD3', 'hg38_ABCD4', 'hg38_ABCE1', 'hg38_ABCF1', 'hg38_ABCF2', 'hg38_ABCF3', 'hg38_ABCG1', 'hg38_ABCG2', 'hg38_ABHD1', 'hg38_ABHD10', 'hg38_ABHD11', 'hg38_ABHD11-AS1', 'hg38_ABHD12', 'hg38_ABHD13', 'hg38_ABHD14A', 'hg38_ABHD14B', 'hg38_ABHD15', 'hg38_ABHD15-AS1', 'hg38_ABHD16A', 'hg38_ABHD16B', 'hg38_ABHD17A', 'hg38_ABHD17B', 'hg38_ABHD17C', 'hg

In [0]:
dot_data = tree.export_graphviz(dt_clf, out_file=None, 
...                      feature_names=feature_names,  
...                      class_names=target_names,  
...                      filled=True, rounded=True,  
...                      special_characters=True)
graph = graphviz.Source(dot_data)
graph.render("model")

'model.pdf'

In [0]:
dot_data_out = tree.export_graphviz(dt_clf_out, out_file=None, 
...                      feature_names=feature_names,  
...                      class_names=target_names,  
...                      filled=True, rounded=True,  
...                      special_characters=True)
graph_out = graphviz.Source(dot_data_out)
graph_out.render("model_outliers")

'model_outliers.pdf'

**Ensamble algorithm - bagging**

In [0]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from random import randrange
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

In [0]:
num_bags = 15
percentage = 0.6

# promeni!!!!!

est_mdl = DecisionTreeClassifier()

bagg_clf = BaggingClassifier(base_estimator=est_mdl, n_estimators=num_bags, max_samples=percentage)#, max_features=percentage)

# bagg_clf = BaggingRegressor(base_estimator=est_mdl, n_estimators=num_bags, max_samples=percentage)
bagg_clf.fit(A_train, b_train)

BaggingClassifier(base_estimator=DecisionTreeClassifier(class_weight=None,
                                                        criterion='gini',
                                                        max_depth=None,
                                                        max_features=None,
                                                        max_leaf_nodes=None,
                                                        min_impurity_decrease=0.0,
                                                        min_impurity_split=None,
                                                        min_samples_leaf=1,
                                                        min_samples_split=2,
                                                        min_weight_fraction_leaf=0.0,
                                                        presort=False,
                                                        random_state=None,
                                                        splitter='best'),
    

In [0]:
b_test_predicted_bagging = bagg_clf.predict(A_test)

In [0]:
print("Bagging test score")
print(bagg_clf.score(A_test, b_test))

Bagging test score
0.9835414301929626


In [0]:
b_train_predicted_bagging = bagg_clf.predict(A_train)

In [0]:
print("Bagging train score")
print(bagg_clf.score(A_train, b_train))

Bagging train score
0.9931873479318735


In [0]:
test_rep_bagging = classification_report(b_test, b_test_predicted_bagging)
print("Bagging test report:\n{}".format(test_rep_bagging))

Bagging test report:
              precision    recall  f1-score   support

          33       0.98      0.99      0.99       977
          34       0.98      0.98      0.98       785

    accuracy                           0.98      1762
   macro avg       0.98      0.98      0.98      1762
weighted avg       0.98      0.98      0.98      1762



In [0]:
train_rep_bagging = classification_report(b_train, b_train_predicted_bagging)
print("Bagging train report:\n{}".format(train_rep_bagging))

Bagging train report:
              precision    recall  f1-score   support

          33       0.99      1.00      0.99      2279
          34       1.00      0.99      0.99      1831

    accuracy                           0.99      4110
   macro avg       0.99      0.99      0.99      4110
weighted avg       0.99      0.99      0.99      4110



In [0]:
test_conf_bagging = confusion_matrix(b_test, b_test_predicted_bagging)
print("Confusion matrix - test:\n{}".format(test_conf_bagging))

Confusion matrix - test:
[[965  12]
 [ 17 768]]


In [0]:
train_conf_bagging = confusion_matrix(b_train, b_train_predicted_bagging)
print("Confusion matrix - train:\n{}".format(train_conf_bagging))

Confusion matrix - train:
[[2273    6]
 [  22 1809]]


In [0]:
num_bags = 15
percentage = 0.6

# promeni!!!!!

est_mdl_out = DecisionTreeClassifier()

bagg_clf_out = BaggingClassifier(base_estimator=est_mdl_out, n_estimators=num_bags, max_samples=percentage)#, max_features=percentage)

# bagg_clf = BaggingRegressor(base_estimator=est_mdl, n_estimators=num_bags, max_samples=percentage)
bagg_clf_out.fit(X_train, y_train)

BaggingClassifier(base_estimator=DecisionTreeClassifier(class_weight=None,
                                                        criterion='gini',
                                                        max_depth=None,
                                                        max_features=None,
                                                        max_leaf_nodes=None,
                                                        min_impurity_decrease=0.0,
                                                        min_impurity_split=None,
                                                        min_samples_leaf=1,
                                                        min_samples_split=2,
                                                        min_weight_fraction_leaf=0.0,
                                                        presort=False,
                                                        random_state=None,
                                                        splitter='best'),
    

In [0]:
y_test_predicted_bagg = bagg_clf_out.predict(X_test)

In [0]:
print("Bagging test score for outliers:")
print(bagg_clf_out.score(X_test, y_test))

Bagging test score for outliers:
0.9730538922155688


In [0]:
y_train_predicted_bagg = bagg_clf_out.predict(X_train)

In [0]:
print("Bagging train score for outliers:")
print(bagg_clf_out.score(X_train, y_train))

Bagging train score for outliers:
0.9922879177377892


In [0]:
test_rep_bagg_out = classification_report(y_test, y_test_predicted_bagg)
print("Bagging test report for outliers:\n{}".format(test_rep_bagg_out))

Bagging test report for outliers:
              precision    recall  f1-score   support

          33       0.96      0.98      0.97       157
          34       0.98      0.97      0.97       177

    accuracy                           0.97       334
   macro avg       0.97      0.97      0.97       334
weighted avg       0.97      0.97      0.97       334



In [0]:
train_rep_bagg_out = classification_report(y_train, y_train_predicted_bagg)
print("Bagging train report for outliers:\n{}".format(train_rep_bagg_out))

Bagging train report for outliers:
              precision    recall  f1-score   support

          33       0.99      0.99      0.99       366
          34       1.00      0.99      0.99       412

    accuracy                           0.99       778
   macro avg       0.99      0.99      0.99       778
weighted avg       0.99      0.99      0.99       778



In [0]:
test_conf_bagging_out = confusion_matrix(y_test, y_test_predicted_bagg)
print("Confusion matrix - test:\n{}".format(test_conf_bagging_out))

Confusion matrix - test:
[[154   3]
 [  6 171]]


In [0]:
train_conf_bagging_out = confusion_matrix(y_train, y_train_predicted_bagg)
print("Confusion matrix - train:\n{}".format(train_conf_bagging_out))

Confusion matrix - train:
[[364   2]
 [  4 408]]


**Ensamble algorithm - voting**

In [0]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB

In [0]:
# clf1 = LogisticRegression()
clf1 = SVC(gamma='scale', kernel='rbf', probability=True)
clf2 = DecisionTreeClassifier()
clf3 = MultinomialNB()
clf4 = SGDClassifier(loss='log')

In [0]:
voting_clf = VotingClassifier(estimators=[('SVM', clf1), ('DTC', clf2), ('MNB', clf3), ('SGD', clf4)], voting='soft', weights=[1,2,0.5,0.5])
voting_clf.fit(A_train, b_train)

VotingClassifier(estimators=[('SVM',
                              SVC(C=1.0, cache_size=200, class_weight=None,
                                  coef0=0.0, decision_function_shape='ovr',
                                  degree=3, gamma='scale', kernel='rbf',
                                  max_iter=-1, probability=True,
                                  random_state=None, shrinking=True, tol=0.001,
                                  verbose=False)),
                             ('DTC',
                              DecisionTreeClassifier(class_weight=None,
                                                     criterion='gini',
                                                     max_depth=None,
                                                     max_features=None,
                                                     max_leaf_...
                                            class_weight=None,
                                            early_stopping=False, epsilon=0.1,
             

In [0]:
b_test_predicted_voting = voting_clf.predict(A_test)

In [0]:
print("Voting test score:")
print(voting_clf.score(A_test, b_test))

Voting score:
0.9835414301929626


In [0]:
b_train_predicted_voting = voting_clf.predict(A_train)

In [0]:
print("Voting train score:")
print(voting_clf.score(A_train, b_train))

Voting train score:
1.0


In [0]:
test_rep_voting = classification_report(b_test, b_test_predicted_voting)
print("Voting test report:\n{}".format(test_rep_voting))

Voting test report:
              precision    recall  f1-score   support

          33       0.98      0.99      0.99       977
          34       0.98      0.98      0.98       785

    accuracy                           0.98      1762
   macro avg       0.98      0.98      0.98      1762
weighted avg       0.98      0.98      0.98      1762



In [0]:
train_rep_voting = classification_report(b_train, b_train_predicted_voting)
print("Voting train report:\n{}".format(train_rep_voting))

Voting train report:
              precision    recall  f1-score   support

          33       1.00      1.00      1.00      2279
          34       1.00      1.00      1.00      1831

    accuracy                           1.00      4110
   macro avg       1.00      1.00      1.00      4110
weighted avg       1.00      1.00      1.00      4110



In [0]:
test_conf_voting = confusion_matrix(b_test, b_test_predicted_voting)
print("Confusion matrix - test:\n{}".format(test_conf_voting))

Confusion matrix - test:
[[963  14]
 [ 15 770]]


In [0]:
train_conf_voting = confusion_matrix(b_train, b_train_predicted_voting)
print("Confusion matrix - train:\n{}".format(train_conf_voting))

Confusion matrix - train:
[[2279    0]
 [   0 1831]]


In [0]:
clf1_out = SVC(gamma='scale', kernel='rbf', probability=True)
clf2_out = DecisionTreeClassifier()
clf3_out = MultinomialNB()
clf4_out = SGDClassifier(loss='log')

In [0]:
voting_clf_out = VotingClassifier(estimators=[('SVM', clf1_out), ('DTC', clf2_out), ('MNB', clf3_out), ('SGD', clf4_out)], voting='soft', weights=[1,2,0.5,0.5])
voting_clf_out.fit(X_train, y_train)

VotingClassifier(estimators=[('SVM',
                              SVC(C=1.0, cache_size=200, class_weight=None,
                                  coef0=0.0, decision_function_shape='ovr',
                                  degree=3, gamma='scale', kernel='rbf',
                                  max_iter=-1, probability=True,
                                  random_state=None, shrinking=True, tol=0.001,
                                  verbose=False)),
                             ('DTC',
                              DecisionTreeClassifier(class_weight=None,
                                                     criterion='gini',
                                                     max_depth=None,
                                                     max_features=None,
                                                     max_leaf_...
                                            class_weight=None,
                                            early_stopping=False, epsilon=0.1,
             

In [0]:
y_train_predicted_voting = voting_clf_out.predict(X_train)

In [0]:
print("Voting train score for outliers:")
print(voting_clf_out.score(X_train, y_train))

Voting train score for outliers:
1.0


In [0]:
y_test_predicted_voting = voting_clf_out.predict(X_test)

In [0]:
print("Voting test score for outliers:")
print(voting_clf_out.score(X_test, y_test))

Voting test score for outliers:
0.9461077844311377


In [0]:
test_rep_voting_out = classification_report(y_test, y_test_predicted_voting)
print("Voting test report for outliers:\n{}".format(test_rep_voting_out))

Voting test report for outliers:
              precision    recall  f1-score   support

          33       0.94      0.95      0.94       157
          34       0.95      0.94      0.95       177

    accuracy                           0.95       334
   macro avg       0.95      0.95      0.95       334
weighted avg       0.95      0.95      0.95       334



In [0]:
train_rep_voting_out = classification_report(y_train, y_train_predicted_voting)
print("Voting train report for outliers:\n{}".format(train_rep_voting_out))

Voting train report for outliers:
              precision    recall  f1-score   support

          33       1.00      1.00      1.00       366
          34       1.00      1.00      1.00       412

    accuracy                           1.00       778
   macro avg       1.00      1.00      1.00       778
weighted avg       1.00      1.00      1.00       778



In [0]:
test_conf_voting_out = confusion_matrix(y_test, y_test_predicted_voting)
print("Confusion matrix - test:\n{}".format(test_conf_voting_out))

Confusion matrix - test:
[[149   8]
 [ 10 167]]


In [0]:
train_conf_voting_out = confusion_matrix(y_train, y_train_predicted_voting)
print("Confusion matrix - train:\n{}".format(train_conf_voting_out))

Confusion matrix - train:
[[366   0]
 [  0 412]]


**Ensabmle algorithm - random forest**

In [0]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [0]:
rf_clf = RandomForestClassifier(n_estimators=45, bootstrap=False)
rf_clf.fit(A_train, b_train)

RandomForestClassifier(bootstrap=False, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=45,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [0]:
b_test_predicted_rf = rf_clf.predict(A_test)

In [0]:
print("Random forest test score:")
print(rf_clf.score(A_test, b_test))

Random forest test score:
0.9631101021566402


In [0]:
b_train_predicted_rf = rf_clf.predict(A_train)

In [0]:
print("Random forest train score:")
print(rf_clf.score(A_train, b_train))

Random forest train score:
1.0


In [0]:
test_rep_rf = classification_report(b_test, b_test_predicted_rf)
print("Random forest test report:\n{}".format(test_rep_rf))

Random forest test report:
              precision    recall  f1-score   support

          33       0.95      0.98      0.97       977
          34       0.97      0.94      0.96       785

    accuracy                           0.96      1762
   macro avg       0.96      0.96      0.96      1762
weighted avg       0.96      0.96      0.96      1762



In [0]:
train_rep_rf = classification_report(b_train, b_train_predicted_rf)
print("Random forest train report:\n{}".format(train_rep_rf))

Random forest train report:
              precision    recall  f1-score   support

          33       1.00      1.00      1.00      2279
          34       1.00      1.00      1.00      1831

    accuracy                           1.00      4110
   macro avg       1.00      1.00      1.00      4110
weighted avg       1.00      1.00      1.00      4110



In [0]:
test_conf_rf = confusion_matrix(b_test, b_test_predicted_rf)
print("Confusion matrix - test:\n{}".format(test_conf_rf))

Confusion matrix - test:
[[958  19]
 [ 46 739]]


In [0]:
train_conf_rf = confusion_matrix(b_train, b_train_predicted_rf)
print("Confusion matrix - train:\n{}".format(train_conf_rf))

Confusion matrix - train:
[[2279    0]
 [   0 1831]]


In [0]:
rf_clf_out = RandomForestClassifier(n_estimators=45, bootstrap=False)
rf_clf_out.fit(X_train, y_train)

RandomForestClassifier(bootstrap=False, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=45,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [0]:
y_test_predicted_rf = rf_clf_out.predict(X_test)

In [0]:
print("Random forest test score for outliers:")
print(rf_clf_out.score(X_test, y_test))

Random forest test score for outliers:
0.937125748502994


In [0]:
y_train_predicted_rf = rf_clf_out.predict(X_train)

In [0]:
print("Random forest train score for outliers:")
print(rf_clf_out.score(X_train, y_train))

Random forest train score for outliers:
1.0


In [0]:
test_rep_rf_out = classification_report(y_test, y_test_predicted_rf)
print("Random forest test report for outliers:\n{}".format(test_rep_rf_out))

Random forest test report for outliers:
              precision    recall  f1-score   support

          33       0.93      0.94      0.93       157
          34       0.94      0.94      0.94       177

    accuracy                           0.94       334
   macro avg       0.94      0.94      0.94       334
weighted avg       0.94      0.94      0.94       334



In [0]:
train_rep_rf_out = classification_report(y_train, y_train_predicted_rf)
print("Random forest train report for outliers:\n{}".format(train_rep_rf_out))

Random forest train report for outliers:
              precision    recall  f1-score   support

          33       1.00      1.00      1.00       366
          34       1.00      1.00      1.00       412

    accuracy                           1.00       778
   macro avg       1.00      1.00      1.00       778
weighted avg       1.00      1.00      1.00       778



In [0]:
test_conf_rf_out = confusion_matrix(y_test, y_test_predicted_rf)
print("Confusion matrix - test:\n{}".format(test_conf_rf_out))

Confusion matrix - test:
[[147  10]
 [ 11 166]]


In [0]:
train_conf_rf_out = confusion_matrix(y_train, y_train_predicted_rf)
print("Confusion matrix - train:\n{}".format(train_conf_rf_out))

Confusion matrix - train:
[[366   0]
 [  0 412]]


**Ensamble algorithm - Ada boost**

In [0]:
from sklearn.ensemble import AdaBoostClassifier

In [0]:
dt_clf = DecisionTreeClassifier(max_depth=3)

ada_clf = AdaBoostClassifier(base_estimator=dt_clf, n_estimators=45)
ada_clf.fit(A_train, b_train)

AdaBoostClassifier(algorithm='SAMME.R',
                   base_estimator=DecisionTreeClassifier(class_weight=None,
                                                         criterion='gini',
                                                         max_depth=3,
                                                         max_features=None,
                                                         max_leaf_nodes=None,
                                                         min_impurity_decrease=0.0,
                                                         min_impurity_split=None,
                                                         min_samples_leaf=1,
                                                         min_samples_split=2,
                                                         min_weight_fraction_leaf=0.0,
                                                         presort=False,
                                                         random_state=None,
                             

In [0]:
b_test_predicted_ada = ada_clf.predict(A_test)

In [0]:
print("Ada boost test score:")
print(ada_clf.score(A_test, b_test))

Ada boost test score:
0.9897843359818388


In [0]:
b_train_predicted_ada = ada_clf.predict(A_train)

In [0]:
print("Ada boost train score:")
print(ada_clf.score(A_train, b_train))

Ada boost train score:
1.0


In [0]:
test_rep_ada = classification_report(b_test, b_test_predicted_ada)
print("Ada boost test report:\n{}".format(test_rep_ada))

Ada boost test report:
              precision    recall  f1-score   support

          33       0.98      0.98      0.98       977
          34       0.98      0.97      0.98       785

    accuracy                           0.98      1762
   macro avg       0.98      0.98      0.98      1762
weighted avg       0.98      0.98      0.98      1762



In [0]:
train_rep_ada = classification_report(b_train, b_train_predicted_ada)
print("Ada boost train report:\n{}".format(train_rep_ada))

Ada boost train report:
              precision    recall  f1-score   support

          33       1.00      1.00      1.00      2279
          34       1.00      1.00      1.00      1831

    accuracy                           1.00      4110
   macro avg       1.00      1.00      1.00      4110
weighted avg       1.00      1.00      1.00      4110



In [0]:
test_conf_ada = confusion_matrix(b_test, b_test_predicted_ada)
print("Confusion matrix - test:\n{}".format(test_conf_ada))

Confusion matrix - test:
[[960  17]
 [ 21 764]]


In [0]:
train_conf_ada = confusion_matrix(b_train, b_train_predicted_ada)
print("Confusion matrix - train:\n{}".format(train_conf_ada))

Confusion matrix - train:
[[2279    0]
 [   0 1831]]


In [0]:
dt_clf_out = DecisionTreeClassifier()

ada_clf_out = AdaBoostClassifier(base_estimator=dt_clf_out, n_estimators=45)
ada_clf_out.fit(X_train, y_train)

AdaBoostClassifier(algorithm='SAMME.R',
                   base_estimator=DecisionTreeClassifier(class_weight=None,
                                                         criterion='gini',
                                                         max_depth=None,
                                                         max_features=None,
                                                         max_leaf_nodes=None,
                                                         min_impurity_decrease=0.0,
                                                         min_impurity_split=None,
                                                         min_samples_leaf=1,
                                                         min_samples_split=2,
                                                         min_weight_fraction_leaf=0.0,
                                                         presort=False,
                                                         random_state=None,
                          

In [0]:
y_test_predicted_ada = ada_clf_out.predict(X_test)

In [0]:
print("Ada boost test score for outliers:")
print(ada_clf_out.score(X_test, y_test))

Ada boost test score for outliers:
0.9491017964071856


In [0]:
y_train_predicted_ada = ada_clf_out.predict(X_train)

In [0]:
print("Ada boost train score for outliers:")
print(ada_clf_out.score(X_train, y_train))

Ada boost train score for outliers:
1.0


In [0]:
test_rep_ada_out = classification_report(y_test, y_test_predicted_ada)
print("Ada boost test report for outliers:\n{}".format(test_rep_ada_out))

Ada boost test report for outliers:
              precision    recall  f1-score   support

          33       0.94      0.95      0.95       157
          34       0.95      0.95      0.95       177

    accuracy                           0.95       334
   macro avg       0.95      0.95      0.95       334
weighted avg       0.95      0.95      0.95       334



In [0]:
train_rep_ada_out = classification_report(y_train, y_train_predicted_ada)
print("Ada boost train report for outliers:\n{}".format(train_rep_ada_out))

Ada boost train report for outliers:
              precision    recall  f1-score   support

          33       1.00      1.00      1.00       366
          34       1.00      1.00      1.00       412

    accuracy                           1.00       778
   macro avg       1.00      1.00      1.00       778
weighted avg       1.00      1.00      1.00       778



In [0]:
test_conf_ada_out = confusion_matrix(y_test, y_test_predicted_ada)
print("Confusion matrix - test:\n{}".format(test_conf_ada_out))

Confusion matrix - test:
[[149   8]
 [  9 168]]


In [0]:
train_conf_ada_out = confusion_matrix(y_train, y_train_predicted_ada)
print("Confusion matrix - train:\n{}".format(train_conf_ada_out))

Confusion matrix - train:
[[366   0]
 [  0 412]]


**Neural network**

In [0]:
from sklearn.neural_network import MLPClassifier

In [0]:
mlp_clf = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(15, 2), activation='logistic')
mlp_clf.fit(A_train, b_train)

/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLPClassifier(activation='logistic', alpha=1e-05, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(15, 2), learning_rate='constant',
              learning_rate_init=0.001, max_iter=200, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=None, shuffle=True, solver='adam', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)

In [0]:
print("MultiLayer Perceptron test score:")
print(mlp_clf.score(A_test, b_test))

MultiLayer Perceptron test score:
0.9954597048808173


In [0]:
b_test_predicted_mlp = mlp_clf.predict(A_test)

In [0]:
print("MultiLayer Perceptron train score:")
print(mlp_clf.score(A_train, b_train))

MultiLayer Perceptron train score:
0.9997566909975669


In [0]:
b_train_predicted_mlp = mlp_clf.predict(A_train)

In [0]:
test_rep_mlp = classification_report(b_test, b_test_predicted_mlp)
print("MultiLayer Perceptron test report:\n{}".format(test_rep_mlp))

MultiLayer Perceptron test report:
              precision    recall  f1-score   support

          33       1.00      1.00      1.00       977
          34       0.99      0.99      0.99       785

    accuracy                           1.00      1762
   macro avg       1.00      1.00      1.00      1762
weighted avg       1.00      1.00      1.00      1762



In [0]:
train_rep_mlp = classification_report(b_train, b_train_predicted_mlp)
print("MultiLayer Perceptron train report:\n{}".format(train_rep_mlp))

MultiLayer Perceptron train report:
              precision    recall  f1-score   support

          33       1.00      1.00      1.00      2279
          34       1.00      1.00      1.00      1831

    accuracy                           1.00      4110
   macro avg       1.00      1.00      1.00      4110
weighted avg       1.00      1.00      1.00      4110



In [0]:
test_conf_mlp = confusion_matrix(b_test, b_test_predicted_mlp)
print("Confusion matrix - test:\n{}".format(test_conf_mlp))

Confusion matrix - test:
[[973   4]
 [  4 781]]


In [0]:
train_conf_mlp = confusion_matrix(b_train, b_train_predicted_mlp)
print("Confusion matrix - train:\n{}".format(train_conf_mlp))

Confusion matrix - train:
[[2279    0]
 [   1 1830]]


In [0]:
mlp_clf_out = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(15, 2), activation='logistic')
mlp_clf_out.fit(X_train, y_train)

MLPClassifier(activation='logistic', alpha=1e-05, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(15, 2), learning_rate='constant',
              learning_rate_init=0.001, max_iter=200, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=None, shuffle=True, solver='lbfgs', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)

In [0]:
y_test_predicted_mlp = mlp_clf_out.predict(X_test)

In [0]:
print("MultiLayer Perceptron test score for outliers:")
print(mlp_clf_out.score(X_test, y_test))

MultiLayer Perceptron test score for outliers:
0.9730538922155688


In [0]:
y_train_predicted_mlp = mlp_clf_out.predict(X_train)

In [0]:
print("MultiLayer Perceptron train score for outliers:")
print(mlp_clf_out.score(X_train, y_train))

MultiLayer Perceptron train score for outliers:
0.9832904884318766


In [0]:
test_rep_mlp_out = classification_report(y_test, y_test_predicted_mlp)
print("MultiLayer Perceptron test report for outliers:\n{}".format(test_rep_mlp_out))

MultiLayer Perceptron test report for outliers:
              precision    recall  f1-score   support

          33       0.96      0.98      0.97       157
          34       0.98      0.97      0.97       177

    accuracy                           0.97       334
   macro avg       0.97      0.97      0.97       334
weighted avg       0.97      0.97      0.97       334



In [0]:
train_rep_mlp_out = classification_report(y_train, y_train_predicted_mlp)
print("MultiLayer Perceptron train report for outliers:\n{}".format(train_rep_mlp_out))

MultiLayer Perceptron train report for outliers:
              precision    recall  f1-score   support

          33       0.98      0.98      0.98       366
          34       0.99      0.98      0.98       412

    accuracy                           0.98       778
   macro avg       0.98      0.98      0.98       778
weighted avg       0.98      0.98      0.98       778



In [0]:
test_conf_mlp_out = confusion_matrix(y_test, y_test_predicted_mlp)
print("Confusion matrix - test:\n{}".format(test_conf_mlp_out))

Confusion matrix - test:
[[154   3]
 [  6 171]]


In [0]:
train_conf_mlp_out = confusion_matrix(y_train, y_train_predicted_mlp)
print("Confusion matrix - train:\n{}".format(train_conf_mlp_out))

Confusion matrix - train:
[[360   6]
 [  7 405]]
